In [1]:
import numpy as np
import torch

from interactive_seg_backend.file_handling import load_image, load_labels
from interactive_seg_backend.main import featurise, train_and_apply, TrainingConfig, FeatureConfig

from vulture.main import (
    get_hr_feats,
    get_dv2_model,
    get_upsampler_and_expr,
)
from vulture.utils import to_numpy
from is_helpers import get_deep_feats, train_model_over_images, apply_model_over_images, eval_preds, BaselineAdditions

from PIL import Image
from skimage.color import label2rgb
import matplotlib.pyplot as plt

from typing import Any, Literal

SEED = 10673
np.random.seed(SEED)
torch.manual_seed(SEED)
DEVICE = "cuda:1"

N CPUS: 110


/home/ronan/miniconda3/envs/dv2/lib/python3.12/site-packages/kornia/feature/lightglue.py:44: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @torch.cuda.amp.custom_fwd(cast_inputs=torch.float32)


In [ ]:
dv2 = get_dv2_model(True, device=DEVICE)

model_path = "../trained_models/fit_reg_f128.pth"
# cfg_path = "../vulture/models/configs/combined_no_shift.json"

upsampler, expr = get_upsampler_and_expr(model_path, None, device=DEVICE)

Using cache found in /home/ronan/.cache/torch/hub/ywyue_FiT3D_main


In [3]:
baselines: list[BaselineAdditions] = ["random", "uniform", "duplicate"]
PATH = "fig_data/is_benchmark"
AllowedDatasets = Literal["Ni_superalloy_SEM", "T_cell_TEM", "Cu_ore_RLM"]

TRAIN_IMG_FNAMES: dict[AllowedDatasets, list[str]] = {"Cu_ore_RLM": ["004", "028", "049", "077"], 
                                                      "Ni_superalloy_SEM": ["000", "001", "005", "007"], 
                                                      "T_cell_TEM": ["000", "005", "007", "026"]
                                                      }

all_preds = {baseline: {} for baseline in baselines}

In [4]:
chosen_dataset = "Ni_superalloy_SEM"
for baseline in baselines:
    feat_cfg = FeatureConfig()

    classical_train_cfg = TrainingConfig(feat_cfg, n_samples=-1, add_dino_features=False, classifier='xgb', classifier_params = {'class_weight': 'balanced'})
    classical_model, _ = train_model_over_images(chosen_dataset, classical_train_cfg, PATH, TRAIN_IMG_FNAMES[chosen_dataset], dv2, upsampler, expr)

    classical_preds = apply_model_over_images(chosen_dataset, classical_train_cfg, classical_model, PATH, dv2, upsampler, expr, verbose=True)
    all_preds[baseline] = classical_preds

Finished featurising
Finished featurising
Finished featurising
Finished featurising
[00/23] - 000.tif
[10/23] - 010.tif
[20/23] - 020.tif
Finished featurising
Finished featurising
Finished featurising
Finished featurising
[00/23] - 000.tif
[10/23] - 010.tif
[20/23] - 020.tif
Finished featurising
Finished featurising
Finished featurising
Finished featurising
[00/23] - 000.tif
[10/23] - 010.tif
[20/23] - 020.tif


In [5]:
for baseline in baselines:
    classical_preds = all_preds[baseline]
    miou_classical, miou_std_classical = eval_preds(chosen_dataset, classical_preds, PATH)
    print(f"======== {baseline} ========")
    print(f"mIoU_classical: {miou_classical:.4f}+/-{miou_std_classical:.4f}\n")

======== random ========
mIoU_classical: 0.5409+/-0.1245

======== uniform ========
mIoU_classical: 0.5422+/-0.1231

======== duplicate ========
mIoU_classical: 0.5411+/-0.1248

